In [1]:
!python -m pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [2]:
# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [3]:
# function to count sequence length
def count_seq_length(df: pd.DataFrame, seq_col: str) -> pd.Series:
    pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
    df[seq_col].replace(pattern, "", inplace= True)
    return df[seq_col].str.len()

In [4]:
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
counts = count_seq_length(pd.read_csv(TRAIN_DATAPATH), "sequence")

In [5]:
# Initialize WANDB
PROJECT = 'custom_plot'
RUN = "run_21"
wandb.init(project = PROJECT, name = RUN)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
hist, edges = np.histogram(counts, bins = 10)
histogram_data = {
    "hist": hist,
    "edges": edges
}
wandb.Histogram(histogram_data)

TypeError: ignored

In [ ]:
# create report
report = wr.Report(project = PROJECT,
                   title = "Retention time report_data",
                   description = "A first try creating a WANDB report using DLOmix")
report.save()

Report(project='custom_plot', entity='master_praktikum', title='Retention time report_data', description='A first try creating a WANDB report using DLOmix', width='readable')

In [ ]:
ENTITY = wandb.apis.PublicApi().default_entity

c = counts.to_list()
data = [[s] for s in c]
table = wandb.Table(data=data, columns=["counts"])
hist = wandb.plot_table(
    vega_spec_name="simple_hist",
    data_table = table,
    fields = "counts")
wandb.log({'my_histogram': hist})

pg_val = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT),
    ],
    panels=[
        wr.CustomChart(
            query = {"summary": ["counts"]},
            chart_name='wandb/histogram/v0',
            chart_fields={'x': '_step'}
        )
    ]
)

report.blocks = report.blocks[:1] + [wr.H1("Data"), pg_val]

report.save()

wandb.finish()

# i dont think its possible to use CustomChart with a vega_spec
# might actually be possible but docu is shit
# everything seems working except query

In [ ]:
c = counts.to_list()
data = [[s] for s in c]
table = wandb.Table(data=data, columns=["counts"])
histogram = wandb.plot.histogram(table, value='height', title='Histogram')
wandb.log({"hist_1" : histogram})
wandb.finish()